<a href="https://colab.research.google.com/github/Chessmag/automatminer/blob/master/Magpie_Termoconductivity_predicted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install plotly
!pip install TPOT
#!pip install matminer

In [2]:
#!git clone https://github.com/Chessmag/matminer.git
#!cd matminer
#!sudo python matminer/setup.py install


In [3]:
!pip install git+https://github.com/Chessmag/matminer.git
#!git clone https://github.com/Chessmag/matminer.git
#!cd automatminer
#!cd automatminer
#!pip install .
#!ls
#!sudo python setup.py install
#!apt-get install matminer

  Cloning https://github.com/Chessmag/matminer.git to /tmp/pip-req-build-hu7r44iu
  Running command git clone -q https://github.com/Chessmag/matminer.git /tmp/pip-req-build-hu7r44iu
Processing /root/.cache/pip/wheels/79/8d/92/41519a1ae6409b90acd325f25aa6869b2451deb24d46c2b0ea/pymatgen-2020.9.14-cp36-cp36m-linux_x86_64.whl
  Created wheel for matminer: filename=matminer-0.6.2-cp36-none-any.whl size=1167159 sha256=5bd7d2058626ead09988899e72a9ed5f7a53bda27e074f99686e102d4a949572
  Stored in directory: /tmp/pip-ephem-wheel-cache-ztli2j83/wheels/10/88/96/715074f4c65072fa378a71be79e8a2d6af0944a011fae7f1ac
Successfully built matminer
ERROR: automatminer 1.0.3.20200727 has requirement pymatgen==2020.01.28, but you'll have pymatgen 2020.9.14 which is incompatible.
  Found existing installation: pymatgen 2020.1.28
    Uninstalling pymatgen-2020.1.28:
      Successfully uninstalled pymatgen-2020.1.28


In [4]:
!pip install automatminer
#!git clone https://github.com/Chessmag/automatminer.git
#!cd automatminer
#!sudo python automatminer/setup.py install


Processing /root/.cache/pip/wheels/42/26/f7/55c561a7d4b18e0850ac7582903ec54ce495a775112f5268fa/pymatgen-2020.1.28-cp36-cp36m-linux_x86_64.whl
ERROR: matminer 0.6.2 has requirement pymatgen>=2020.4.29, but you'll have pymatgen 2020.1.28 which is incompatible.
  Found existing installation: pymatgen 2020.9.14
    Uninstalling pymatgen-2020.9.14:
      Successfully uninstalled pymatgen-2020.9.14


In [5]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import plotly.express as px
from matminer.data_retrieval.retrieve_MP import MPDataRetrieval

from matminer.featurizers.base import BaseFeaturizer
from matminer.featurizers.composition import ElementProperty
from matminer.utils.data import PymatgenData
from matminer.utils.data import MagpieData
from pymatgen import Composition
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score
from matminer.figrecipes.plot import PlotlyFig
from sklearn.ensemble import RandomForestRegressor
from automatminer.pipeline import MatPipe

In [6]:
from google.colab import files
uploaded = files.upload()

Saving ThermoDB.csv to ThermoDB (1).csv


In [29]:
import io
df_mp = pd.read_csv(io.BytesIO(uploaded['ThermoDB.csv']))


In [30]:
print ('Number of bulk moduli extracted = {}'.format(len(df_mp)))
#df_mp.drop([0:(len(df_mp)-10)], inplace=True)
df_mp.head()
df_mp.describe()

# explore columns
df_mp.head()

df_mp["composition"] = df_mp['Composition'].map(lambda x: Composition(x))

#dataset = MagpieData()
descriptors = ['Number', 'MendeleevNumber', 'AtomicWeight',
                          'MeltingT', 'Column', 'Row', 'CovalentRadius',
                          'Electronegativity', 'NsValence', 'NpValence',
                          'NdValence', 'NfValence', 'NValence', 'NsUnfilled',
                          'NpUnfilled', 'NdUnfilled', 'NfUnfilled', 'NUnfilled',
                          'GSvolume_pa', 'GSbandgap', 'GSmagmom',
                          'SpaceGroupNumber']
stats = ['minimum', 'maximum', 'range', 'mean', 'avg_dev',
                       'mode']

ep = ElementProperty(data_source=MagpieData(), features=descriptors, stats=stats)
print(ep)
df_mp = ep.featurize_dataframe(df_mp, "composition")
#df_mp['PymatgenData mean vickers_hardness'] = df_mp['PymatgenData mean vickers_hardness']*0.009807
#df_mp['PymatgenData std_dev vickers_hardness'] = df_mp['PymatgenData std_dev vickers_hardness']*0.009807
#df_mp = df_mp[df_mp['PymatgenData mean vickers_hardness'] < 50]
#Remove NaN values
df_mp = df_mp.dropna()

df_mp.head()


# target output column
y = df_mp['Thermal_conductivity'].values

# possible descriptor columns
X_cols = [c for c in df_mp.columns
          if c not in ['Thermal_conductivity', 'Thermal_expansion', 'heat_capacity','Unnamed: 0', 'Composition', 'composition']]
print(X_cols)


ElementProperty:   0%|          | 0/5534 [00:00<?, ?it/s]

Number of bulk moduli extracted = 5534
ElementProperty(data_source=<matminer.utils.data.MagpieData object at 0x7f050504d2b0>,
                features=['Number', 'MendeleevNumber', 'AtomicWeight',
                          'MeltingT', 'Column', 'Row', 'CovalentRadius',
                          'Electronegativity', 'NsValence', 'NpValence',
                          'NdValence', 'NfValence', 'NValence', 'NsUnfilled',
                          'NpUnfilled', 'NdUnfilled', 'NfUnfilled', 'NUnfilled',
                          'GSvolume_pa', 'GSbandgap', 'GSmagmom',
                          'SpaceGroupNumber'],
                stats=['minimum', 'maximum', 'range', 'mean', 'avg_dev',
                       'mode'])


ElementProperty: 100%|██████████| 5534/5534 [00:07<00:00, 737.41it/s]


['density', 'vpa', 'MagpieData minimum Number', 'MagpieData maximum Number', 'MagpieData range Number', 'MagpieData mean Number', 'MagpieData avg_dev Number', 'MagpieData mode Number', 'MagpieData minimum MendeleevNumber', 'MagpieData maximum MendeleevNumber', 'MagpieData range MendeleevNumber', 'MagpieData mean MendeleevNumber', 'MagpieData avg_dev MendeleevNumber', 'MagpieData mode MendeleevNumber', 'MagpieData minimum AtomicWeight', 'MagpieData maximum AtomicWeight', 'MagpieData range AtomicWeight', 'MagpieData mean AtomicWeight', 'MagpieData avg_dev AtomicWeight', 'MagpieData mode AtomicWeight', 'MagpieData minimum MeltingT', 'MagpieData maximum MeltingT', 'MagpieData range MeltingT', 'MagpieData mean MeltingT', 'MagpieData avg_dev MeltingT', 'MagpieData mode MeltingT', 'MagpieData minimum Column', 'MagpieData maximum Column', 'MagpieData range Column', 'MagpieData mean Column', 'MagpieData avg_dev Column', 'MagpieData mode Column', 'MagpieData minimum Row', 'MagpieData maximum Row

In [31]:
df_mp.drop(['Composition', 'composition','Unnamed: 0','Thermal_conductivity','Thermal_expansion', 'heat_capacity'], axis='columns', inplace=True)
print(df_mp.columns)


Index(['density', 'vpa', 'MagpieData minimum Number', 'MagpieData maximum Number', 'MagpieData range Number', 'MagpieData mean Number', 'MagpieData avg_dev Number', 'MagpieData mode Number', 'MagpieData minimum MendeleevNumber', 'MagpieData maximum MendeleevNumber',
       ...
       'MagpieData range GSmagmom', 'MagpieData mean GSmagmom', 'MagpieData avg_dev GSmagmom', 'MagpieData mode GSmagmom', 'MagpieData minimum SpaceGroupNumber', 'MagpieData maximum SpaceGroupNumber', 'MagpieData range SpaceGroupNumber', 'MagpieData mean SpaceGroupNumber', 'MagpieData avg_dev SpaceGroupNumber', 'MagpieData mode SpaceGroupNumber'], dtype='object', length=134)


In [32]:
#df_mp.drop(['Composition'], axis='columns', inplace=True)
#df_mp.to_excel("TiVhardness_highprecmet.xlsx")
#df_HEA.to_csv("TiVNbMoWhardness_highprecmet.csv")
#from google.colab import files
#files.download('TiVhardness_highprecmet.xlsx') 
#files.download('TiVNbMoWhardness_highprecmet.csv') 
print (df_mp[5:10])


     density        vpa  MagpieData minimum Number  MagpieData maximum Number  MagpieData range Number  MagpieData mean Number  MagpieData avg_dev Number  MagpieData mode Number  MagpieData minimum MendeleevNumber  MagpieData maximum MendeleevNumber  MagpieData range MendeleevNumber  MagpieData mean MendeleevNumber  MagpieData avg_dev MendeleevNumber  MagpieData mode MendeleevNumber  MagpieData minimum AtomicWeight  MagpieData maximum AtomicWeight  MagpieData range AtomicWeight  MagpieData mean AtomicWeight  MagpieData avg_dev AtomicWeight  MagpieData mode AtomicWeight  MagpieData minimum MeltingT  MagpieData maximum MeltingT  MagpieData range MeltingT  MagpieData mean MeltingT  MagpieData avg_dev MeltingT  MagpieData mode MeltingT  MagpieData minimum Column  MagpieData maximum Column  MagpieData range Column  MagpieData mean Column  MagpieData avg_dev Column  MagpieData mode Column  MagpieData minimum Row  MagpieData maximum Row  MagpieData range Row  MagpieData mean Row  \
6    9.468

In [33]:

X = df_mp.to_numpy(dtype='float')

print("Possible descriptors are: {}".format(X_cols))



lr = LinearRegression()

lr.fit(X, y)

# get fit statistics
print ('R2 = ' + str(round(lr.score(X, y), 3)))
print ('RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y, y_pred=lr.predict(X))))

Possible descriptors are: ['density', 'vpa', 'MagpieData minimum Number', 'MagpieData maximum Number', 'MagpieData range Number', 'MagpieData mean Number', 'MagpieData avg_dev Number', 'MagpieData mode Number', 'MagpieData minimum MendeleevNumber', 'MagpieData maximum MendeleevNumber', 'MagpieData range MendeleevNumber', 'MagpieData mean MendeleevNumber', 'MagpieData avg_dev MendeleevNumber', 'MagpieData mode MendeleevNumber', 'MagpieData minimum AtomicWeight', 'MagpieData maximum AtomicWeight', 'MagpieData range AtomicWeight', 'MagpieData mean AtomicWeight', 'MagpieData avg_dev AtomicWeight', 'MagpieData mode AtomicWeight', 'MagpieData minimum MeltingT', 'MagpieData maximum MeltingT', 'MagpieData range MeltingT', 'MagpieData mean MeltingT', 'MagpieData avg_dev MeltingT', 'MagpieData mode MeltingT', 'MagpieData minimum Column', 'MagpieData maximum Column', 'MagpieData range Column', 'MagpieData mean Column', 'MagpieData avg_dev Column', 'MagpieData mode Column', 'MagpieData minimum Row

In [34]:
print(df_mp[0:100])

       density         vpa  MagpieData minimum Number  MagpieData maximum Number  MagpieData range Number  MagpieData mean Number  MagpieData avg_dev Number  MagpieData mode Number  MagpieData minimum MendeleevNumber  MagpieData maximum MendeleevNumber  MagpieData range MendeleevNumber  MagpieData mean MendeleevNumber  MagpieData avg_dev MendeleevNumber  MagpieData mode MendeleevNumber  MagpieData minimum AtomicWeight  MagpieData maximum AtomicWeight  MagpieData range AtomicWeight  MagpieData mean AtomicWeight  MagpieData avg_dev AtomicWeight  MagpieData mode AtomicWeight  MagpieData minimum MeltingT  MagpieData maximum MeltingT  MagpieData range MeltingT  MagpieData mean MeltingT  MagpieData avg_dev MeltingT  MagpieData mode MeltingT  MagpieData minimum Column  MagpieData maximum Column  MagpieData range Column  MagpieData mean Column  MagpieData avg_dev Column  MagpieData mode Column  MagpieData minimum Row  MagpieData maximum Row  MagpieData range Row  MagpieData mean Row  \
0     8

In [35]:
print(X)


[[  8.67484     40.003       83.         ...  12.           0.
   12.        ]
 [ 10.3692      30.58638889  37.         ... 171.25         9.625
  166.        ]
 [ 10.2956      30.18194444  19.         ... 171.25         9.625
  166.        ]
 ...
 [  3.31235      6.021325     6.         ... 194.           0.
  194.        ]
 [  3.49045      5.7141       6.         ... 194.           0.
  194.        ]
 [  3.49723      5.703        6.         ... 194.           0.
  194.        ]]


In [36]:
print(X_cols)

['density', 'vpa', 'MagpieData minimum Number', 'MagpieData maximum Number', 'MagpieData range Number', 'MagpieData mean Number', 'MagpieData avg_dev Number', 'MagpieData mode Number', 'MagpieData minimum MendeleevNumber', 'MagpieData maximum MendeleevNumber', 'MagpieData range MendeleevNumber', 'MagpieData mean MendeleevNumber', 'MagpieData avg_dev MendeleevNumber', 'MagpieData mode MendeleevNumber', 'MagpieData minimum AtomicWeight', 'MagpieData maximum AtomicWeight', 'MagpieData range AtomicWeight', 'MagpieData mean AtomicWeight', 'MagpieData avg_dev AtomicWeight', 'MagpieData mode AtomicWeight', 'MagpieData minimum MeltingT', 'MagpieData maximum MeltingT', 'MagpieData range MeltingT', 'MagpieData mean MeltingT', 'MagpieData avg_dev MeltingT', 'MagpieData mode MeltingT', 'MagpieData minimum Column', 'MagpieData maximum Column', 'MagpieData range Column', 'MagpieData mean Column', 'MagpieData avg_dev Column', 'MagpieData mode Column', 'MagpieData minimum Row', 'MagpieData maximum Row

In [37]:
lr.predict([X[103]])

array([-5.0418638])

In [38]:
X.shape

(5533, 134)

In [39]:
# Use 10-fold cross validation (90% training, 10% test)
crossvalidation = KFold(n_splits=20, shuffle=True, random_state=1)

# compute cross validation scores for random forest model
scores = cross_val_score(lr, X, y,
                         cv=crossvalidation, n_jobs=4)
rmse_scores = [np.sqrt(abs(s)) for s in scores]

print ('Cross-validation results:')
print ('Folds: %i, mean RMSE: %.3f' % (len(scores), np.mean(np.abs(rmse_scores))))

Cross-validation results:
Folds: 20, mean RMSE: 0.656


In [40]:

fig = px.scatter(x=y, y=lr.predict(X))
fig.show()

In [41]:
pf = PlotlyFig(x_title='DFT (MP) bulk modulus (GPa)',
               y_title='Predicted bulk modulus (GPa)',
               #plot_title='Linear regression',
               #plot_mode='offline',
               #margin_left=150,
               #textsize=35,
               ticksize=30,
               filename="lr_regression.html")

# a line to represent a perfect model with 1:1 prediction
xy_params = {'x_col': [0, 400],
             'y_col': [0, 400],
             'color': 'black',
             'mode': 'lines',
             'legend': None,
             'text': None,
             'size': None}

pf.xy((y, lr.predict(X)))
           #size=3,
           #marker_outline_width=0.5,
           #text=df_mp['pretty_formula'],
           #add_xy_plot=[xy_params])


In [42]:
rf = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                       criterion='mse', max_depth=None,
                                       max_features=0.7500000000000002,
                                       max_leaf_nodes=None, max_samples=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=7, min_samples_split=11,
                                       min_weight_fraction_leaf=0.0,
                                       n_estimators=200, 
                                       oob_score=False, random_state=None,
                                       verbose=0, warm_start=False, n_jobs=16)
rf.fit(X, y)
print ('RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y, y_pred=rf.predict(X))))

# compute cross validation scores for random forest model
scores = cross_val_score(rf, X, y, scoring='neg_mean_squared_error', cv=crossvalidation, n_jobs=16)
rmse_scores = [np.sqrt(abs(s)) for s in scores]
#for s in scores:
  #print(math.sqrt(abs(s)))
print ('Cross-validation results:')
print ('Folds: %i, mean RMSE: %.3f' % (len(scores), np.mean(np.abs(rmse_scores))))

RMSE = 6.942
Cross-validation results:
Folds: 20, mean RMSE: 7.995


In [43]:
#scora = cross_val_score(rf, X, y, scoring='neg_mean_absolute_error', cv=crossvalidation, n_jobs=16)
#mae_scores = [s for s in scora]

#print ('Folds: %i, mean MAE: %.3f' % (len(scores), np.mean(np.abs(mae_scores))))


In [44]:
fig = px.scatter(x=y, y=rf.predict(X))
fig.show()

In [45]:
#rf.predict()

In [46]:
importances = rf.feature_importances_
X_cols = np.asarray(X_cols)
indices = np.argsort(importances)[::-1]

In [47]:
print(indices)
print(X_cols)
print(importances)
print(X_cols[indices])
print(importances[indices])

[  1  39   3  63 113  15 107 104 111  19  23   0   7 109  24  16  22  20
  89  44  40 112 115  25  96 114  78   4  48  41  18  42  30  10 108 131
  46  49  90  77  64  76   5  47  26  17 132  75  95 119 116   6 127  11
  21 120  13  29  12  60  65  38 125  66 110  45  74  35   9  86  36 129
  43  53 130   8 117   2  34  14  31 133  59  28  83 126  79  94  67  58
 106  55  87  72 105  91  54 118 121  97  88  84  27  62  93 123  61  56
 124 128 101  92  71  81  50 102  32  85  52  57  37  99  70  69  33 100
 122  82  73 103  68  98  80  51]
['density' 'vpa' 'MagpieData minimum Number' 'MagpieData maximum Number'
 'MagpieData range Number' 'MagpieData mean Number'
 'MagpieData avg_dev Number' 'MagpieData mode Number'
 'MagpieData minimum MendeleevNumber' 'MagpieData maximum MendeleevNumber'
 'MagpieData range MendeleevNumber' 'MagpieData mean MendeleevNumber'
 'MagpieData avg_dev MendeleevNumber' 'MagpieData mode MendeleevNumber'
 'MagpieData minimum AtomicWeight' 'MagpieData maximum Atom

In [48]:
df = pd.DataFrame(dict(
    r=importances[indices],
    theta=X_cols[indices]))
fig = px.line_polar(df, r='r', theta='theta', line_close=True)
fig.show()
print(X_cols[indices])

['vpa' 'MagpieData maximum CovalentRadius' 'MagpieData maximum Number'
 'MagpieData maximum NdValence' 'MagpieData mean GSvolume_pa'
 'MagpieData maximum AtomicWeight' 'MagpieData mean NUnfilled'
 'MagpieData minimum NUnfilled' 'MagpieData maximum GSvolume_pa'
 'MagpieData mode AtomicWeight' 'MagpieData mean MeltingT' 'density'
 'MagpieData mode Number' 'MagpieData mode NUnfilled'
 'MagpieData avg_dev MeltingT' 'MagpieData range AtomicWeight'
 'MagpieData range MeltingT' 'MagpieData minimum MeltingT'
 'MagpieData mean NpUnfilled' 'MagpieData minimum Electronegativity'
 'MagpieData range CovalentRadius' 'MagpieData range GSvolume_pa'
 'MagpieData mode GSvolume_pa' 'MagpieData mode MeltingT'
 'MagpieData avg_dev NdUnfilled' 'MagpieData avg_dev GSvolume_pa'
 'MagpieData avg_dev NValence' 'MagpieData range Number'
 'MagpieData avg_dev Electronegativity' 'MagpieData mean CovalentRadius'
 'MagpieData avg_dev AtomicWeight' 'MagpieData avg_dev CovalentRadius'
 'MagpieData avg_dev Column' 'Magp

In [49]:
import plotly.graph_objects as go
fig = go.Figure([go.Bar(x=X_cols[indices], y=importances[indices])])
fig.show()

In [67]:
from google.colab import files
uploaded = files.upload()


Saving Refractfor_calc5.csv to Refractfor_calc5 (2).csv


In [68]:
import io
df_HEA = pd.read_csv(io.BytesIO(uploaded['Refractfor_calc5.csv']))
#df_HEA.drop([0], inplace=True)
#print(df_HEA)

In [69]:
# properties are the materials attributes we want
# See https://github.com/materialsproject/mapidoc for available properties you can specify
#properties = ['pretty_formula', 'spacegroup.symbol', 'elasticity.K_VRH', 'formation_energy_per_atom', 'band_gap',
              #'e_above_hull', 'density', 'volume', 'nsites']

# get the data!

#print ('Number of bulk moduli extracted = {}'.format(len(df_HEA)))


In [70]:
#df_mp.drop([0:(len(df_mp)-10)], inplace=True)
#df_HEA.describe()

# explore columns
#df_HEA.head()


df_HEA["composition"] = df_HEA['Composition'].map(lambda x: Composition(x))



In [71]:
#dataset = PymatgenData()
descriptors = ['Number', 'MendeleevNumber', 'AtomicWeight',
                          'MeltingT', 'Column', 'Row', 'CovalentRadius',
                          'Electronegativity', 'NsValence', 'NpValence',
                          'NdValence', 'NfValence', 'NValence', 'NsUnfilled',
                          'NpUnfilled', 'NdUnfilled', 'NfUnfilled', 'NUnfilled',
                          'GSvolume_pa', 'GSbandgap', 'GSmagmom',
                          'SpaceGroupNumber']
stats = ['minimum', 'maximum', 'range', 'mean', 'avg_dev',
                       'mode']



#print(ep)
#df_mp = ep.featurize_dataframe(df_mp, "composition")

#print(df_HEA)
df_HEA1 = df_HEA

In [72]:
ed = ElementProperty(data_source=MagpieData(), features=descriptors, stats=stats)
#print(ed)

In [73]:
df_HEA = ed.featurize_dataframe(df_HEA, "composition")
#df_HEA['PymatgenData mean vickers_hardness'] = df_HEA['PymatgenData mean vickers_hardness']*0.009807
#df_HEA['PymatgenData std_dev vickers_hardness'] = df_HEA['PymatgenData std_dev vickers_hardness']*0.009807
#Remove NaN values
df_HEA = df_HEA.dropna()

df_HEA.head()

ElementProperty: 100%|██████████| 126/126 [00:00<00:00, 545.67it/s]


,Unnamed: 0,Composition,density,vpa,composition,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,MagpieData avg_dev Number,MagpieData mode Number,MagpieData minimum MendeleevNumber,MagpieData maximum MendeleevNumber,MagpieData range MendeleevNumber,MagpieData mean MendeleevNumber,MagpieData avg_dev MendeleevNumber,MagpieData mode MendeleevNumber,MagpieData minimum AtomicWeight,MagpieData maximum AtomicWeight,MagpieData range AtomicWeight,MagpieData mean AtomicWeight,MagpieData avg_dev AtomicWeight,MagpieData mode AtomicWeight,MagpieData minimum MeltingT,MagpieData maximum MeltingT,MagpieData range MeltingT,MagpieData mean MeltingT,MagpieData avg_dev MeltingT,MagpieData mode MeltingT,MagpieData minimum Column,MagpieData maximum Column,MagpieData range Column,MagpieData mean Column,MagpieData avg_dev Column,MagpieData mode Column,MagpieData minimum Row,MagpieData maximum Row,MagpieData range Row,MagpieData mean Row,MagpieData avg_dev Row,MagpieData mode Row,MagpieData minimum CovalentRadius,MagpieData maximum CovalentRadius,MagpieData range CovalentRadius,MagpieData mean CovalentRadius,MagpieData avg_dev CovalentRadius,MagpieData mode CovalentRadius,MagpieData minimum Electronegativity,MagpieData maximum Electronegativity,MagpieData range Electronegativity,MagpieData mean Electronegativity,MagpieData avg_dev Electronegativity,MagpieData mode Electronegativity,MagpieData minimum NsValence,MagpieData maximum NsValence,MagpieData range NsValence,MagpieData mean NsValence,MagpieData avg_dev NsValence,MagpieData mode NsValence,MagpieData minimum NpValence,MagpieData maximum NpValence,MagpieData range NpValence,MagpieData mean NpValence,MagpieData avg_dev NpValence,MagpieData mode NpValence,MagpieData minimum NdValence,MagpieData maximum NdValence,MagpieData range NdValence,MagpieData mean NdValence,MagpieData avg_dev NdValence,MagpieData mode NdValence,MagpieData minimum NfValence,MagpieData maximum NfValence,MagpieData range NfValence,MagpieData mean NfValence,MagpieData avg_dev NfValence,MagpieData mode NfValence,MagpieData minimum NValence,MagpieData maximum NValence,MagpieData range NValence,MagpieData mean NValence,MagpieData avg_dev NValence,MagpieData mode NValence,MagpieData minimum NsUnfilled,MagpieData maximum NsUnfilled,MagpieData range NsUnfilled,MagpieData mean NsUnfilled,MagpieData avg_dev NsUnfilled,MagpieData mode NsUnfilled,MagpieData minimum NpUnfilled,MagpieData maximum NpUnfilled,MagpieData range NpUnfilled,MagpieData mean NpUnfilled,MagpieData avg_dev NpUnfilled,MagpieData mode NpUnfilled,MagpieData minimum NdUnfilled,MagpieData maximum NdUnfilled,MagpieData range NdUnfilled,MagpieData mean NdUnfilled,MagpieData avg_dev NdUnfilled,MagpieData mode NdUnfilled,MagpieData minimum NfUnfilled,MagpieData maximum NfUnfilled,MagpieData range NfUnfilled,MagpieData mean NfUnfilled,MagpieData avg_dev NfUnfilled,MagpieData mode NfUnfilled,MagpieData minimum NUnfilled,MagpieData maximum NUnfilled,MagpieData range NUnfilled,MagpieData mean NUnfilled,MagpieData avg_dev NUnfilled,MagpieData mode NUnfilled,MagpieData minimum GSvolume_pa,MagpieData maximum GSvolume_pa,MagpieData range GSvolume_pa,MagpieData mean GSvolume_pa,MagpieData avg_dev GSvolume_pa,MagpieData mode GSvolume_pa,MagpieData minimum GSbandgap,MagpieData maximum GSbandgap,MagpieData range GSbandgap,MagpieData mean GSbandgap,MagpieData avg_dev GSbandgap,MagpieData mode GSbandgap,MagpieData minimum GSmagmom,MagpieData maximum GSmagmom,MagpieData range GSmagmom,MagpieData mean GSmagmom,MagpieData avg_dev GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber
0,1,Ti20V20Cr20Zr20Nb20,6.39,17.41,"(Ti, V, Cr, Zr, Nb)",22.0,41.0,19.0,30.0,8.40,22.0,43.0,49.0,6.0,45.8,1.84,43.0,47.867,92.90638,45.03938,66.986996,20.062555,47.867,1941.0,2750.

In [74]:

df_HEA.drop(['Unnamed: 0', 'Composition', 'composition'], axis='columns', inplace=True)
print(df_HEA.columns)


KeyError: ignored

In [ ]:
X = df_HEA.to_numpy(dtype='float')

In [ ]:
rf.predict(X)
print(len(rf.predict(X)))


In [ ]:
df_HEA["Thermal_conductivity"] = rf.predict(X)
df_HEA['Composition'] = df_HEA1['Composition']
#print(df_HEA.head)


In [ ]:
df_HEA.drop(['density', 'vpa', 'MagpieData minimum Number','MagpieData maximum Number','MagpieData range Number','MagpieData mean Number','MagpieData avg_dev Number','MagpieData mode Number','MagpieData minimum MendeleevNumber','MagpieData maximum MendeleevNumber','MagpieData range MendeleevNumber','MagpieData mean MendeleevNumber','MagpieData avg_dev MendeleevNumber','MagpieData mode MendeleevNumber','MagpieData minimum AtomicWeight','MagpieData maximum AtomicWeight','MagpieData range AtomicWeight','MagpieData mean AtomicWeight','MagpieData avg_dev AtomicWeight','MagpieData mode AtomicWeight','MagpieData minimum MeltingT','MagpieData maximum MeltingT','MagpieData range MeltingT','MagpieData mean MeltingT','MagpieData avg_dev MeltingT','MagpieData mode MeltingT','MagpieData minimum Column','MagpieData maximum Column','MagpieData range Column','MagpieData mean Column','MagpieData avg_dev Column','MagpieData mode Column','MagpieData minimum Row','MagpieData maximum Row','MagpieData range Row','MagpieData mean Row','MagpieData avg_dev Row','MagpieData mode Row','MagpieData minimum CovalentRadius','MagpieData maximum CovalentRadius','MagpieData range CovalentRadius','MagpieData mean CovalentRadius','MagpieData avg_dev CovalentRadius','MagpieData mode CovalentRadius','MagpieData minimum Electronegativity','MagpieData maximum Electronegativity','MagpieData range Electronegativity','MagpieData mean Electronegativity','MagpieData avg_dev Electronegativity','MagpieData mode Electronegativity','MagpieData minimum NsValence','MagpieData maximum NsValence','MagpieData range NsValence','MagpieData mean NsValence','MagpieData avg_dev NsValence','MagpieData mode NsValence','MagpieData minimum NpValence','MagpieData maximum NpValence','MagpieData range NpValence','MagpieData mean NpValence','MagpieData avg_dev NpValence','MagpieData mode NpValence','MagpieData minimum NdValence','MagpieData maximum NdValence','MagpieData range NdValence','MagpieData mean NdValence','MagpieData avg_dev NdValence','MagpieData mode NdValence','MagpieData minimum NfValence','MagpieData maximum NfValence','MagpieData range NfValence','MagpieData mean NfValence','MagpieData avg_dev NfValence','MagpieData mode NfValence','MagpieData minimum NValence','MagpieData maximum NValence','MagpieData range NValence','MagpieData mean NValence','MagpieData avg_dev NValence','MagpieData mode NValence','MagpieData minimum NsUnfilled','MagpieData maximum NsUnfilled','MagpieData range NsUnfilled','MagpieData mean NsUnfilled','MagpieData avg_dev NsUnfilled','MagpieData mode NsUnfilled','MagpieData minimum NpUnfilled','MagpieData maximum NpUnfilled','MagpieData range NpUnfilled','MagpieData mean NpUnfilled','MagpieData avg_dev NpUnfilled','MagpieData mode NpUnfilled','MagpieData minimum NdUnfilled','MagpieData maximum NdUnfilled','MagpieData range NdUnfilled','MagpieData mean NdUnfilled','MagpieData avg_dev NdUnfilled','MagpieData mode NdUnfilled','MagpieData minimum NfUnfilled','MagpieData maximum NfUnfilled','MagpieData range NfUnfilled','MagpieData mean NfUnfilled','MagpieData avg_dev NfUnfilled','MagpieData mode NfUnfilled','MagpieData minimum NUnfilled','MagpieData maximum NUnfilled','MagpieData range NUnfilled','MagpieData mean NUnfilled','MagpieData avg_dev NUnfilled','MagpieData mode NUnfilled','MagpieData minimum GSvolume_pa','MagpieData maximum GSvolume_pa','MagpieData range GSvolume_pa','MagpieData mean GSvolume_pa','MagpieData avg_dev GSvolume_pa','MagpieData mode GSvolume_pa','MagpieData minimum GSbandgap','MagpieData maximum GSbandgap','MagpieData range GSbandgap','MagpieData mean GSbandgap','MagpieData avg_dev GSbandgap','MagpieData mode GSbandgap','MagpieData minimum GSmagmom','MagpieData maximum GSmagmom','MagpieData range GSmagmom','MagpieData mean GSmagmom','MagpieData avg_dev GSmagmom','MagpieData mode GSmagmom','MagpieData minimum SpaceGroupNumber','MagpieData maximum SpaceGroupNumber','MagpieData range SpaceGroupNumber','MagpieData mean SpaceGroupNumber','MagpieData avg_dev SpaceGroupNumber','MagpieData mode SpaceGroupNumber'], axis='columns', inplace=True)

In [ ]:
df_HEA.to_excel("Refractfor_calc.xlsx")
df_HEA.to_csv("Refractfor_calc.csv")
from google.colab import files
files.download('Refractfor_calc.xlsx') 
files.download('Refractfor_calc.csv') 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')